<a href="https://colab.research.google.com/github/sumithkumar07/Text-_summarization_by_using_transformer/blob/main/Text__summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opendatasets

In [ ]:
!pip install transformers --upgrade

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [ ]:
train_data = pd.read_csv("/content/samsum-dataset-text-summarization/samsum-train.csv")

validation_data = pd.read_csv("/content/samsum-dataset-text-summarization/samsum-validation.csv")

In [ ]:
train_data = train_data.sample(n=6000,random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)

In [ ]:
import re

def clean_text(text):

    if isinstance(text, str):
        text = re.sub(r'\r\n', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'<.*?>', '', text)
        text = text.strip().lower()
    else:
        text = str(text)
    return text

train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)

train_data.head()

In [ ]:
# Tokenization

tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["dialogue"], padding = 'max_length', truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding = 'max_length', truncation=True, max_length=150)
    inputs['labels'] = targets['input_ids']
    return inputs



train_dataset = train_data.apply(preprocess_function, axis=1)
validation_data = validation_data.apply(preprocess_function, axis=1)

In [ ]:
train_dataset[0]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 50,
    save_steps = 500,
    eval_steps = 50,

    eval_strategy = "epoch",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = validation_data
)

trainer.train()